In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.stats import pearsonr

In [20]:
df_train = pd.read_csv('train.csv')
ans = df_train['correlation']

In [114]:
def corr(X, Y):
    return pearsonr(X, Y)[0]


def corrCV(image, i):
    # plt.figure(figsize = (45,30))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # plt.imshow(erosion)
    # return
    th, threshed = cv2.threshold(image, i, 255, cv2.THRESH_BINARY)
    kernel = np.ones((1, 1),np.uint8)
    erosion = cv2.erode(threshed,kernel,iterations = 1)
    contours, hierarchy = cv2.findContours(erosion, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # print(len(contours))
    X, Y = [], []
    for c in contours:  # find center point
        M = cv2.moments(c)
        if M['m00']:
            cX = M['m10']/M['m00']
            cY = M['m01']/M['m00']
            X.append(cX)
            Y.append(cY)
        # cv2.circle(image, (int(cX), int(cY)), 0, (0, 0, 255), -1)
    result = -corr(X, Y)
    # cv2.drawContours(image, contours, -1, (255,0, 0), 2)
    # plt.imshow(image)
    return result


def corrCV2(image, i):
    hMin = 100  # Hue minimum
    sMin = 70  # Saturation minimum
    vMin = 60  # Value minimum (Also referred to as brightness)
    hMax = 220 # Hue maximum
    sMax = 255 # Saturation maximum
    vMax = 170 # Value maximum
    # Set the minimum and max HSV values to display in the output image using numpys' array function. We need the numpy array since OpenCVs' inRange function will use those.
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])
    # Create HSV Image and threshold it into the proper range.
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Converting color space from BGR to HSV
    mask = cv2.inRange(hsv, lower, upper)
    # Create a mask based on the lower and upper range, using the new HSV image# Create the output image, using the mask created above. This will perform the removal of all unneeded colors, but will keep a black background.
    output = cv2.bitwise_and(image, image, mask=mask)# Add an alpha channel, and update the output image variable
    *_, alpha = cv2.split(output)
    dst = cv2.merge((output.astype('uint8'), alpha.astype('uint8')))
    kernel = np.ones((4, 4),np.uint8)
    dst = cv2.morphologyEx(dst, cv2.MORPH_CLOSE, kernel)
    # plt.figure(figsize = (45,30))
    # plt.imshow(dst)
    th, threshed = cv2.threshold(dst, i, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(threshed, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # print(len(contours))
    X, Y = [], []
    for c in contours:  # find center point
        M = cv2.moments(c)
        if M['m00']:
            cX = M['m10']/M['m00']
            cY = M['m01']/M['m00']
            X.append(cX)
            Y.append(cY)
        # cv2.circle(image, (int(cX), int(cY)), 0, (0, 0, 255), -1)
    result = -corr(X, Y)
    # cv2.drawContours(image, contours, -1, (255,0, 0), 2)
    # plt.imshow(image)
    return result



# Find optimal threshold i
# for i in range(255):
#     results = []
#     for path in range(1000):
#         try:
#             result = corrCV(cv2.imread(os.path.join('train', f'{path}.jpg'), 3), i)
#             results.append(result)
#             if pearsonr(ans, results)[0] > 0.990375:
#                 print(pearsonr(ans, results)[0])
#                 print(i)
#         except:
#             pass

In [115]:
df_test = pd.read_csv('submission.csv')
results = []

for path in range(1000):
    result = corrCV2(cv2.imread(os.path.join('test', f'{path}.jpg'), 3), 105)
    results.append(result)
    break

# df_test['correlation'] = results
# df_test.to_csv('submission.csv', index=False)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\contours.cpp:197: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'


In [41]:
# Connect to graders
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'C:/Users/alien/Documents/PyCharm Projects/Cyberthon 2021/pyctfsglib.py')
import pyctfsglib as ctfsg
import random

USER_TOKEN = "WrlLCkymxwtgFwRHZsdmKfSwcdqIpnqoXEtRkciVRZJfBJUgcEJoxVZjNTQRdqkR" # You need to fill this up
GRADER_URL = random.choice([
  "http://chals.cyberthon22t.ctf.sg:50301/",
  "http://chals.cyberthon22t.ctf.sg:50302/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)
grader.submitFile('submission.csv')

DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm CV_CORRELATED @637aa1e3bc7f
ProofOfWork Challenge =>  ('CTFSGRBad90ed58d58d00e3fac651622fb8033c', 22)
ProofOfWork Answer Found! =>  9957800


'{"challenge":{"name":"Correlated"},"id":"cl295vabafmu50832qqev1on1","status":"PARTIALLY_CORRECT","multiplier":0.9745,"submittedBy":{"username":"hci-69"},"createdAt":"2022-04-21T15:31:04Z"}'